# Mean Variance Optimization

- Import data and annualize the mea of monthly returns as well as the volatility of monthly returns with a scaling of $\sqrt(12)$

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import matplotlib as mpl
import seaborn as sns
import scipy as scs
import math

In [2]:
plt.style.use("seaborn")
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [9]:
file_path = "C:/Users/dcste/OneDrive/Portfolio_Theory/multi_asset_etf_data.xlsx"
description = pd.read_excel(file_path,sheet_name = "descriptions")
description

,ticker,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
0,SPY,SPDR S&P 500,ETF,USD,101107853,358229114880,The Trust seeks to achieve its investment obje...
1,EFA,iShares MSCI EAFE ETF,ETF,USD,33352872,43798241280,The fund generally will invest at least 80% of...
2,EEM,iShares MSCI Emerging Index Fun,ETF,USD,47539498,25870192640,The fund generally will invest at least 80% of...
3,PSP,Invesco Global Listed Private E,ETF,USD,120371,171932880,The fund generally will invest at least 90% of...
4,QAI,IQ Hedge MultiIQ Hedge Multi-St,ETF,USD,138713,707315584,"The fund is a ""fund of funds"" which means it i..."
5,HYG,iShares iBoxx $ High Yield Corp,ETF,USD,48935762,12276870144,The underlying index is a rules-based index co...
6,DBC,Invesco DB Commodity Index Trac,ETF,USD,2314311,3708376064,The fund pursues its investment objective by i...
7,IYR,iShares U.S. Real Estate ETF,ETF,USD,12097258,4077254400,The fund seeks to track the investment results...
8,IEF,iShares 7-10 Year Treasury Bond,ETF,USD,7992450,23017226240,The underlying index measures the performance ...
9,BWX,SPDR Bloomberg International Tr,ETF,USD,369873,809217792,"The fund generally invests substantially all, ..."


In [18]:
excess_rt = pd.read_excel(file_path, sheet_name = "excess returns")
excess_rt


,Date,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
0,2009-04-30,0.008439,-0.001554,0.155028,0.114636,0.137906,-0.028006,0.295597,0.229649,0.022329,0.098792,-0.018505
1,2009-05-31,0.054144,0.163134,0.159872,0.132390,0.029027,-0.020301,0.023199,0.054364,0.028337,0.058925,0.020438
2,2009-06-30,0.004549,-0.026858,-0.023094,-0.014649,0.032917,-0.006171,-0.025462,0.044850,-0.004035,-0.001254,0.001383
3,2009-07-31,0.031312,0.018595,0.110173,0.100442,0.069218,0.008344,0.105826,0.143274,0.015353,0.074633,0.000906
4,2009-08-31,0.007193,-0.040800,-0.013571,0.044595,-0.017404,0.007200,0.131503,0.032977,-0.004586,0.036504,0.007977
...,...,...,...,...,...,...,...,...,...,...,...,...
156,2022-04-30,-0.069406,0.056699,-0.061061,-0.067101,-0.041513,-0.041992,-0.041014,-0.125389,-0.033108,-0.087479,-0.021540
157,2022-05-31,0.004797,0.045469,0.005472,0.019297,0.015637,0.005521,-0.045096,0.014422,-0.004688,0.001595,-0.010585
158,2022-06-30,-0.045689,-0.074247,-0.050823,-0.086913,-0.069745,-0.007881,-0.068158,-0.131723,-0.032928,-0.081707,-0.030402
159,2022-07-31,0.019562,-0.020776,-0.004373,0.050807,0.066107,0.028733,0.087725,0.108080,0.017940,0.091206,0.042217


In [38]:
# Scaling excess monthly return to yearly return
annualized_mean = pd.DataFrame((excess_rt.mean()*12), columns = ["Annual Return"])



C:\Users\dcste\AppData\Local\Temp\ipykernel_25948\2834274450.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  annualized_mean = pd.DataFrame((excess_rt.mean()*12), columns = ["Annual Return"])


,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,0.006132,0.004704,0.009122,0.007369,0.003783,0.001599,0.006005,0.008928,0.002458,0.004906,0.002142
DBC,0.004704,0.032639,0.019652,0.016995,0.007374,-0.004344,0.009592,0.019481,0.004773,0.012961,0.000710
EEM,0.009122,0.019652,0.036891,0.026403,0.012507,-0.002724,0.021296,0.033540,0.007497,0.020523,0.002155
EFA,0.007369,0.016995,0.026403,0.026341,0.011133,-0.002550,0.020455,0.032728,0.006705,0.020621,0.001609
HYG,0.003783,0.007374,0.012507,0.011133,0.008046,-0.000492,0.012404,0.016434,0.003366,0.009900,0.001339
IEF,0.001599,-0.004344,-0.002724,-0.002550,-0.000492,0.003527,-0.000185,-0.003205,-0.000144,-0.002318,0.001907
IYR,0.006005,0.009592,0.021296,0.020455,0.012404,-0.000185,0.034130,0.030600,0.005696,0.019898,0.002952
PSP,0.008928,0.019481,0.033540,0.032728,0.016434,-0.003205,0.030600,0.049183,0.009040,0.029122,0.002518
QAI,0.002458,0.004773,0.007497,0.006705,0.003366,-0.000144,0.005696,0.009040,0.002418,0.005970,0.000949
SPY,0.004906,0.012961,0.020523,0.020621,0.009900,-0.002318,0.019898,0.029122,0.005970,0.021100,0.001466
